In [1]:
from bsddb3 import db
database = db.DB() #handle for Berkeley DB database
DB_File = "fruit.db"
# database.open(DB_File ,None, db.DB_HASH, db.DB_CREATE)
database.open(DB_File ,None, db.DB_BTREE, db.DB_CREATE)
# database.open(DB_File ,None, db.DB_QUEUE, db.DB_CREATE)
# database.open(DB_File ,None, db.DB_RECNO, db.DB_CREATE)
# The arguments correspond to (fileName, database name within the file for multiple
# databases, database type, flag to create database)

In [2]:
curs = database.cursor()

In [3]:
# insertion of (key, value) pair using the cursor:
#The arguments corresponds to (key, value, flag to insert key-value pair )
curs.put(b"apple", "red", db.DB_KEYFIRST)
# insertion using the database object’s put method
database.put(b"pear", "green")

In [4]:
# a function to populate fruit.db
def addData():
    database.put(b"pear", "green")
    database.put(b"pear", "yellow")
    database.put(b"orange", "orange")
    database.put(b"apple", "green")
    database.put(b"apple", "red")
    database.put(b"apple", "yellow")
addData()    
#use the cursor to define a function that prints everything in the database
def printAll():
    iter = curs.first()
    while iter:
        print(iter)
        iter = curs.next()
printAll()

(b'apple', b'yellow')
(b'orange', b'orange')
(b'pear', b'yellow')


In [5]:
# using the database object’s get method: only retrieves the value
result = database.get(b'pear')
print(result)

b'yellow'


In [6]:
# using the cursor object’s set method:
result = curs.set(b'apple')
#Moves the cursor to the specified key in the database and return the associated key/value pair.
print(result)

(b'apple', b'yellow')


In [7]:
# remove using the cursor – deletes the key-value pair currently referenced by the cursor:
print("deleting",curs.current(),"\n") #Returns the key/value pair currently referenced by the cursor
curs.delete()
print("This is what's left in the table:")
printAll()

deleting (b'apple', b'yellow') 

This is what's left in the table:
(b'orange', b'orange')
(b'pear', b'yellow')


In [8]:
# closing the database so we can open a new database with duplicates allowed
curs.close()
database.close()

#### Now we enable duplicates

In [9]:
from bsddb3 import db
database = db.DB()
database.set_flags(db.DB_DUP)
DB_File = "fruits_with_duplicates.db"
database.open(DB_File ,None, db.DB_HASH, db.DB_CREATE)
curs = database.cursor()

In [10]:
# a function to populate fruit.db
def addData():
    database.put(b"pear", "green")
    database.put(b"pear", "yellow")
    database.put(b"orange", "orange")
    database.put(b"apple", "green")
    database.put(b"apple", "red")
    database.put(b"apple", "yellow")
    database.put(b"plum", "purple")
    database.put(b"plum", "yellow-green")
addData()    
#use the cursor to define a function that prints everything in the database
def printAll():
    iter = curs.first()
    while iter:
        print(iter)
        iter = curs.next()
printAll()

(b'apple', b'green')
(b'apple', b'red')
(b'apple', b'yellow')
(b'orange', b'orange')
(b'pear', b'green')
(b'pear', b'yellow')
(b'plum', b'purple')
(b'plum', b'yellow-green')


In [11]:
#Duplicate key-value pairs could be traversed using cursor
# prints no. of k-v pairs that have the same key (for the key which the cursor is pointing to)
print(curs.first()) #try cursor.last()
print(curs.count())
# prints the next k-v pair if it is a duplicate
print(curs.next_dup())
#returns ‘None’ if the next k-v pair of the database is not a duplicate data record for the
# current key/value pair

(b'apple', b'green')
3
(b'apple', b'red')


### Exercise 1
Iterating through k-v pair including duplicates:

In [12]:
curs = database.cursor()
iter = curs.first()
while (iter):
    print(curs.count())
    #prints no. of rows that have the same key for the current key-value pair referred by cursor.
    print(iter)
    #iterating through duplicates:
    dup = curs.next_dup()
    while(dup!=None):
        print(dup)
        dup = curs.next_dup()
        iter = curs.next()

3
(b'apple', b'green')
(b'apple', b'red')
(b'apple', b'yellow')
2
(b'pear', b'green')
(b'pear', b'yellow')
2
(b'plum', b'purple')
(b'plum', b'yellow-green')


In [ ]:
from bsddb3 import db
DB_File = "students.db"
database = db.DB()
database.set_flags(db.DB_DUP) #declare duplicates allowed before you create the database
database.open(DB_File,None, db.DB_BTREE, db.DB_CREATE)
curs = database.cursor()

while(True):
    name = input("Enter a student Name to look up: ")
    if(name == "q"): #Termination Condition
        break
    
    result = curs.set(name.encode("utf-8")) 
    #In the presence of duplicate key values, result will be set on the first data item for the given key. 
   
    if(result != None):
        print("List of students with this name and their marks:")
        print("Name: " + str(result[0].decode("utf-8")) + ", Mark: " + str(result[1].decode("utf-8")))
        
        #iterating through duplicates:
        dup = curs.next_dup()
        while(dup != None):
            print("Name: " + str(dup[0].decode("utf-8")) + ", Mark: " + str(dup[1].decode("utf-8")))
            dup = curs.next_dup()
    else:
        print("No Entry Found.")
            
    ToBeAdded = input("Do you want to insert the input name into the database?(Enter y for yes) ")
    
    if(ToBeAdded == "y"):
        database.put(name.encode("utf-8"), input("Insert Mark: "))

curs.close()
database.close()